## 資料處理

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import bisect

import matplotlib.pyplot as plt
%matplotlib inline  

from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from tqdm import tqdm

# pd.options.display.max_rows = 4000
pd.options.display.max_columns = 4000
# pd.options.display.max_seq_items = 2000

In [ ]:
training_df = pd.read_csv('training-set.csv', header=None, names=['FileID', 'label'])
test_df = pd.read_csv('testing-set.csv', header=None, names=['FileID', 'label'])
train_exception_df = pd.read_csv('exception_train.txt', header=None, names=['FileID'])
test_exception_df = pd.read_csv('exception_testing.txt', header=None, names=['FileID'])

training_df = training_df[~training_df['FileID'].isin(train_exception_df['FileID'])]
test_df = test_df[~test_df['FileID'].isin(test_exception_df['FileID'])]

training_df = training_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

print(training_df.shape)
print(test_df.shape)

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
skf = StratifiedKFold(n_splits=5)

training_df['fold'] = None
count = 0
for train_index, test_index in skf.split(training_df.drop('label', axis=1), training_df['label']):
    #print("TRAIN:", train_index, "TEST:", test_index)
    #print(test_index.tolist())
    #print(train_ans_df.iloc[test_index])
    training_df.loc[test_index, 'fold'] = count
    count = count + 1

In [ ]:
five_folds = [[[0, 1, 2, 3], [4]], [[0, 1, 2, 4], [3]], [[0, 1, 3, 4], [2]], [[0, 2, 3, 4], [1]], [[1, 2, 3, 4], [0]]]

In [ ]:
from functools import reduce

temp_dfs = []

for i in range(0, 5):
    temp_dfs.append(pd.read_csv('from_table/customer_virus_ratio_fold'+str(i)+'.csv').set_index('t1_CustomerID'))

# print(temp_dfs[0].head())
# print(temp_dfs[1].head())
# print(temp_dfs[0].add(temp_dfs[1], fill_value=0).head())

customer_virus_ratio_dfs = []
customer_virus_ratio_df_groups = []
for idx, fold in enumerate(five_folds):
    merge_folds = fold[0]
    temp_df = reduce(lambda left, right: left.add(right, fill_value=0), \
                     [temp_dfs[merge_folds[0]], temp_dfs[merge_folds[1]], temp_dfs[merge_folds[2]], temp_dfs[merge_folds[3]]])
    temp_df['ratio'] = temp_df['virus_event_count'] / temp_df['total_event_count']
    #print(temp_df[['virus_event_count', 'total_event_count']])
    customer_virus_ratio_dfs.append(temp_df.reset_index())
    customer_virus_ratio_df_groups.append(customer_virus_ratio_dfs[idx].groupby('t1_CustomerID'))
    #print(customer_virus_ratio_dfs[idx])
    print(idx)
    print(fold)
    

temp_dfs = []

for i in range(0, 5):
    temp_dfs.append(pd.read_csv('from_table/product_virus_ratio_fold'+str(i)+'.csv').set_index('t1_ProductID'))

# print(temp_dfs[0].head())
# print(temp_dfs[1].head())
# print(temp_dfs[0].add(temp_dfs[1], fill_value=0).head())

product_virus_ratio_dfs = []
product_virus_ratio_df_groups = []
for idx, fold in enumerate(five_folds):
    merge_folds = fold[0]
    temp_df = reduce(lambda left, right: left.add(right, fill_value=0), \
                     [temp_dfs[merge_folds[0]], temp_dfs[merge_folds[1]], temp_dfs[merge_folds[2]], temp_dfs[merge_folds[3]]])
    temp_df['ratio'] = temp_df['virus_event_count'] / temp_df['total_event_count']
    #print(temp_df[['virus_event_count', 'total_event_count']])
    product_virus_ratio_dfs.append(temp_df.reset_index())
    product_virus_ratio_df_groups.append(product_virus_ratio_dfs[idx].groupby('t1_ProductID'))
    #print(product_virus_ratio_dfs[idx])
    print(idx)
    print(fold)
    

temp_dfs = []

for i in range(0, 5):
    customer_virus_ratio_df = pd.read_csv('from_table/customer_virus_ratio_fold'+str(i)+'.csv')
    customer_file_count_df = pd.read_csv('from_table/customer_file_count_fold'+str(i)+'.csv')
    
    customer_virus_ratio_df['t2_CustomerID'] = customer_virus_ratio_df['t1_CustomerID']
    customer_virus_ratio_df = customer_virus_ratio_df.drop('t1_CustomerID', axis=1)
    
    customer_file_count_df = customer_file_count_df.merge(customer_virus_ratio_df.drop('total_event_count', axis=1), on='t2_CustomerID', how='left')
    customer_file_count_df = customer_file_count_df.fillna(0)
    temp_dfs.append(customer_file_count_df.set_index('t2_CustomerID'))

# print(temp_dfs[0].head())
# print(temp_dfs[1].head())
# print(temp_dfs[0].add(temp_dfs[1], fill_value=0).head())

customer_file_count_dfs = []
for idx, fold in enumerate(five_folds):
    merge_folds = fold[0]
    temp_df = reduce(lambda left, right: left.add(right, fill_value=0), \
                     [temp_dfs[merge_folds[0]], temp_dfs[merge_folds[1]], temp_dfs[merge_folds[2]], temp_dfs[merge_folds[3]]])
    #print(temp_df[['virus_event_count', 'total_event_count']])
    temp_df['ratio'] = temp_df['virus_event_count'] / (temp_df['total_event_count']+.000000001)
    customer_file_count_dfs.append(temp_df.reset_index())
    #print(product_virus_ratio_dfs[idx])
    print(idx)
    print(fold)

In [ ]:
training_df['affect_user_count'] = None
training_df['total_event_count'] = None
training_df['each_user_event_avg'] = None
training_df['each_user_event_std'] = None
training_df['each_user_event_rstd'] = None

training_df['first_time_occur'] = None
training_df['last_time_occur'] = None
training_df['time_duration'] = None

training_df['first_24h_count'] = None
training_df['first_24h_ratio'] = None
training_df['first_24h_affect_user_count'] = None
training_df['first_24h_affect_user_ratio'] = None
training_df['first_24h_each_user_event_avg'] = None

for i in range(0, 10):
    training_df['bin_'+str(i)+'_count'] = None
    training_df['bin_'+str(i)+'_ratio'] = None
    training_df['bin_'+str(i)+'_affect_user_count'] = None
    training_df['bin_'+str(i)+'_affect_user_ratio'] = None
    training_df['bin_'+str(i)+'_each_user_event_avg'] = None

training_df['customer_spread_time_mean'] = None
training_df['customer_spread_time_std'] = None
training_df['customer_spread_time_rstd'] = None

training_df['event_diff_time_mean'] = None
training_df['event_diff_time_median'] = None
training_df['event_diff_time_std'] = None
training_df['event_diff_time_rstd'] = None

training_df['customer_virus_ratio_avg'] = None
training_df['customer_virus_ratio_min'] = None
training_df['customer_virus_ratio_max'] = None
training_df['customer_virus_ratio_median'] = None
training_df['customer_virus_ratio_std'] = None
training_df['customer_virus_ratio_rstd'] = None  

training_df['product_virus_ratio_avg'] = None
training_df['product_virus_ratio_min'] = None
training_df['product_virus_ratio_max'] = None
training_df['product_virus_ratio_median'] = None
training_df['product_virus_ratio_std'] = None
training_df['product_virus_ratio_rstd'] = None  

training_df['customer_has_static_ratio'] = None
training_df['customer_has_static_b3_ratio'] = None
training_df['customer_has_static_count'] = None
training_df['customer_has_static_b3_count'] = None
training_df['customer_virus_ratio_wavg'] = None
training_df['customer_virus_zero_count'] = None  
training_df['customer_virus_zero_ratio'] = None  
training_df['customer_virus_zero_ratio2'] = None  

training_df['hour_ratio_mean'] = None
training_df['hour_ratio_median'] = None
training_df['hour_ratio_std'] = None
training_df['hour_ratio_rstd'] = None
training_df['hour_ratio_max'] = None
training_df['hour_ratio_min'] = None
training_df['hour_occupy_ratio'] = None
training_df['hour_mean'] = None

training_df['dayofweek_ratio_mean'] = None
training_df['dayofweek_ratio_median'] = None
training_df['dayofweek_ratio_std'] = None
training_df['dayofweek_ratio_rstd'] = None
training_df['dayofweek_ratio_max'] = None
training_df['dayofweek_ratio_min'] = None
training_df['dayofweek_occupy_ratio'] = None
training_df['dayofweek_mean'] = None

training_df['bin_ratio_mean'] = None
training_df['bin_ratio_median'] = None
training_df['bin_ratio_std'] = None
training_df['bin_ratio_rstd'] = None
training_df['bin_ratio_max'] = None
training_df['bin_ratio_min'] = None
training_df['bin_occupy_ratio'] = None

training_df['event_diff_time_mean2'] = None
training_df['event_diff_time_median2'] = None
training_df['event_diff_time_std2'] = None
training_df['event_diff_time_rstd2'] = None
training_df['event_diff_time_max2'] = None
training_df['event_diff_time_min2'] = None
training_df['event_diff_time_lessone_ratio2'] = None

training_df['all_event_diff_time_mean'] = None
training_df['all_event_diff_time_median'] = None
training_df['all_event_diff_time_std'] = None
training_df['all_event_diff_time_rstd'] = None
training_df['all_event_diff_time_max'] = None
training_df['all_event_diff_time_min'] = None
training_df['all_event_diff_time_lessone_ratio'] = None

In [ ]:
for idx, data in tqdm(training_df.iterrows(), total=len(training_df)):
    temp_df = pd.read_csv('file_log/'+data['FileID']+'.csv', header=None, names=['FileID', 'CustomerID', 'QueryTS', 'ProductID'])
    total_event_count = len(temp_df)
    training_df.loc[idx, 'affect_user_count'] = len(temp_df['CustomerID'].unique())
    training_df.loc[idx, 'total_event_count'] = total_event_count
    training_df.loc[idx, 'each_user_event_avg'] = training_df.loc[idx, 'total_event_count'] / training_df.loc[idx, 'affect_user_count']
    training_df.loc[idx, 'each_user_event_std'] = temp_df.groupby('CustomerID').count()['QueryTS'].std()
    training_df.loc[idx, 'each_user_event_rstd'] = training_df.loc[idx, 'each_user_event_std'] / training_df.loc[idx, 'each_user_event_avg']
    
    time_start = temp_df['QueryTS'].min()
    time_end = temp_df['QueryTS'].max()
    time_range = time_end - time_start
    
    training_df.loc[idx, 'first_time_occur'] = time_start
    training_df.loc[idx, 'last_time_occur'] = time_end
    training_df.loc[idx, 'time_duration'] = time_range
    
    the_flag = ( (temp_df['QueryTS']>=time_start)&(temp_df['QueryTS']<=time_start+86400) )
    the_24h_count = np.sum(the_flag)
    training_df.loc[idx, 'first_24h_count'] = the_24h_count
    training_df.loc[idx, 'first_24h_ratio'] = the_24h_count / total_event_count
    training_df.loc[idx, 'first_24h_affect_user_count'] = len(temp_df[the_flag]['CustomerID'].unique())
    training_df.loc[idx, 'first_24h_affect_user_ratio'] = training_df.loc[idx, 'first_24h_affect_user_count'] / training_df.loc[idx, 'affect_user_count']
    training_df.loc[idx, 'first_24h_each_user_event_avg'] = the_24h_count / (training_df.loc[idx, 'first_24h_affect_user_count']+.000000001)
    
    time_step = time_range/10.0
    for i in range(0, 10):
        the_flag = ( (temp_df['QueryTS']>=time_start+time_step*i)&(temp_df['QueryTS']<time_start+time_step*(i+1)+0.5) )
        the_bin_count = np.sum(the_flag)
        training_df.loc[idx, 'bin_'+str(i)+'_count'] = the_bin_count
        training_df.loc[idx, 'bin_'+str(i)+'_ratio'] = the_bin_count / total_event_count
        training_df.loc[idx, 'bin_'+str(i)+'_affect_user_count'] = len(temp_df[the_flag]['CustomerID'].unique())
        training_df.loc[idx, 'bin_'+str(i)+'_affect_user_ratio'] = training_df.loc[idx, 'bin_'+str(i)+'_affect_user_count'] / training_df.loc[idx, 'affect_user_count']
        training_df.loc[idx, 'bin_'+str(i)+'_each_user_event_avg'] = the_bin_count / (training_df.loc[idx, 'bin_'+str(i)+'_affect_user_count']+.000000001)
    
    the_static_df = temp_df.groupby('CustomerID').min().sort_values(['QueryTS'], ascending=True)['QueryTS'].diff().fillna(0)
    training_df.loc[idx, 'customer_spread_time_mean'] = the_static_df.mean()
    training_df.loc[idx, 'customer_spread_time_std'] = the_static_df.std()
    training_df.loc[idx, 'customer_spread_time_rstd'] = training_df.loc[idx, 'customer_spread_time_std'] / (training_df.loc[idx, 'customer_spread_time_mean']+.000000001)
    
    the_static_df = temp_df.sort_values(['CustomerID', 'QueryTS'], ascending=True).groupby(['CustomerID'])['QueryTS'].diff().fillna(0)
    training_df.loc[idx, 'event_diff_time_mean'] = the_static_df.mean()
    training_df.loc[idx, 'event_diff_time_median'] = the_static_df.median()
    training_df.loc[idx, 'event_diff_time_std'] = the_static_df.std()
    training_df.loc[idx, 'event_diff_time_rstd'] = training_df.loc[idx, 'event_diff_time_std'] / (training_df.loc[idx, 'event_diff_time_mean']+.000000001)
    
    temp_df['datetime'] = pd.to_datetime(temp_df['QueryTS'], unit='s')
    temp_df['hour'] = temp_df['datetime'].dt.hour
    temp_df['dayofweek'] = temp_df['datetime'].dt.dayofweek
    
    hour_ratio_df = temp_df[['FileID', 'hour']].groupby('hour').count()/temp_df.shape[0]
    training_df.loc[idx, 'hour_ratio_mean'] = hour_ratio_df['FileID'].mean()
    training_df.loc[idx, 'hour_ratio_median'] = hour_ratio_df['FileID'].median()
    training_df.loc[idx, 'hour_ratio_std'] = hour_ratio_df['FileID'].std()
    training_df.loc[idx, 'hour_ratio_rstd'] = hour_ratio_df['FileID'].std() / (hour_ratio_df['FileID'].mean()+.000000001)
    training_df.loc[idx, 'hour_ratio_max'] = hour_ratio_df['FileID'].max()
    training_df.loc[idx, 'hour_ratio_min'] = hour_ratio_df['FileID'].min()
    training_df.loc[idx, 'hour_occupy_ratio'] = temp_df[['FileID', 'hour']].groupby('hour').count().shape[0]/24.0
    training_df.loc[idx, 'hour_mean'] = temp_df['hour'].mean()
    
    dayofweek_ratio_df = temp_df[['FileID', 'dayofweek']].groupby('dayofweek').count()/temp_df.shape[0]
    training_df.loc[idx, 'dayofweek_ratio_mean'] = dayofweek_ratio_df['FileID'].mean()
    training_df.loc[idx, 'dayofweek_ratio_median'] = dayofweek_ratio_df['FileID'].median()
    training_df.loc[idx, 'dayofweek_ratio_std'] = dayofweek_ratio_df['FileID'].std()
    training_df.loc[idx, 'dayofweek_ratio_rstd'] = dayofweek_ratio_df['FileID'].std() / (dayofweek_ratio_df['FileID'].mean()+.000000001)
    training_df.loc[idx, 'dayofweek_ratio_max'] = dayofweek_ratio_df['FileID'].max()
    training_df.loc[idx, 'dayofweek_ratio_min'] = dayofweek_ratio_df['FileID'].min()
    training_df.loc[idx, 'dayofweek_occupy_ratio'] = temp_df[['FileID', 'dayofweek']].groupby('dayofweek').count().shape[0]/7.0
    training_df.loc[idx, 'dayofweek_mean'] = temp_df['dayofweek'].mean()
    
    bin_ratio_array = training_df.loc[idx, ['bin_0_ratio', 'bin_1_ratio', 'bin_2_ratio', 'bin_3_ratio', 'bin_4_ratio', \
                            'bin_5_ratio', 'bin_6_ratio', 'bin_7_ratio', 'bin_8_ratio', 'bin_9_ratio']].values
    training_df.loc[idx, 'bin_ratio_mean'] = np.mean(bin_ratio_array)
    training_df.loc[idx, 'bin_ratio_median'] = np.median(bin_ratio_array)
    training_df.loc[idx, 'bin_ratio_std'] = np.std(bin_ratio_array)
    training_df.loc[idx, 'bin_ratio_rstd'] = np.std(bin_ratio_array) / (np.mean(bin_ratio_array)+.000000001)
    training_df.loc[idx, 'bin_ratio_max'] = np.max(bin_ratio_array)
    training_df.loc[idx, 'bin_ratio_min'] = np.min(bin_ratio_array)
    training_df.loc[idx, 'bin_occupy_ratio'] = np.sum(bin_ratio_array>0)/10.0
    
    the_static_df = temp_df.sort_values(['CustomerID', 'QueryTS'], ascending=True).groupby(['CustomerID'])['QueryTS'].diff().dropna()
    if the_static_df.shape[0] > 0:
        training_df.loc[idx, 'event_diff_time_mean2'] = the_static_df.mean()
        training_df.loc[idx, 'event_diff_time_median2'] = the_static_df.median()
        training_df.loc[idx, 'event_diff_time_std2'] = the_static_df.std()
        training_df.loc[idx, 'event_diff_time_rstd2'] = training_df.loc[idx, 'event_diff_time_std2'] / (training_df.loc[idx, 'event_diff_time_mean2']+.000000001)
        training_df.loc[idx, 'event_diff_time_max2'] = the_static_df.max()
        training_df.loc[idx, 'event_diff_time_min2'] = the_static_df.min()
        training_df.loc[idx, 'event_diff_time_lessone_ratio2'] = np.sum(the_static_df<=1) / len(the_static_df)
    
    the_static_df = temp_df.sort_values(['QueryTS'], ascending=True)['QueryTS'].diff().dropna()
    if the_static_df.shape[0] > 0:
        training_df.loc[idx, 'all_event_diff_time_mean'] = the_static_df.mean()
        training_df.loc[idx, 'all_event_diff_time_median'] = the_static_df.median()
        training_df.loc[idx, 'all_event_diff_time_std'] = the_static_df.std()
        training_df.loc[idx, 'all_event_diff_time_rstd'] = training_df.loc[idx, 'all_event_diff_time_std'] / (training_df.loc[idx, 'all_event_diff_time_mean']+.000000001)
        training_df.loc[idx, 'all_event_diff_time_max'] = the_static_df.max()
        training_df.loc[idx, 'all_event_diff_time_min'] = the_static_df.min()
        training_df.loc[idx, 'all_event_diff_time_lessone_ratio'] = np.sum(the_static_df<=1) / len(the_static_df)

#     print(training_df.loc[idx])
# #     print(temp_df)
# #     print(len(temp_df['CustomerID'].unique()))
# #     print(data)
#     if idx==2:
#         break

In [ ]:
training_five_folds = []

for fold_idx, fold in enumerate(reversed(five_folds)):
    customer_virus_ratio_df_group = customer_virus_ratio_df_groups[fold_idx]
    product_virus_ratio_df_group = product_virus_ratio_df_groups[fold_idx]
    customer_file_count_df = customer_file_count_dfs[fold_idx].sort_values('t2_CustomerID')
    customer_file_count_array = customer_file_count_df['t2_CustomerID'].tolist()
    customer_file_count_array.sort()
    
    
    fold_training = training_df[training_df['fold'].isin(fold[1])].copy()
    
    for data_idx, data in tqdm(fold_training.iterrows(), total=len(fold_training)):
        temp_df = pd.read_csv('file_log/'+data['FileID']+'.csv', header=None, names=['FileID', 'CustomerID', 'QueryTS', 'ProductID'])
        unique_customers = temp_df['CustomerID'].unique().tolist()
        unique_products = temp_df['ProductID'].unique().tolist()
        
        temp_ratio = []
        temp_count_index = []
        for customer_id in unique_customers:
            try:
                found_ratio = customer_virus_ratio_df_group.get_group(customer_id).iloc[0]
                if found_ratio['total_event_count'] >= 3:
                    temp_ratio.append(found_ratio['ratio'])
            except:
                temp_ratio.append(0)
                
            try:
                found_count_index = bisect.bisect(customer_file_count_array, customer_id)
                if customer_file_count_array[found_count_index-1] == customer_id:
                    temp_count_index.append(found_count_index-1)
            except:
                pass
                
        if len(temp_ratio) > 0:        
            fold_training.loc[data_idx, 'customer_virus_ratio_avg'] = np.mean(temp_ratio)
            fold_training.loc[data_idx, 'customer_virus_ratio_min'] = np.min(temp_ratio)
            fold_training.loc[data_idx, 'customer_virus_ratio_max'] = np.max(temp_ratio)
            fold_training.loc[data_idx, 'customer_virus_ratio_median'] = np.median(temp_ratio)
            fold_training.loc[data_idx, 'customer_virus_ratio_std'] = np.std(temp_ratio)
            fold_training.loc[data_idx, 'customer_virus_ratio_rstd'] = np.std(temp_ratio) / (np.mean(temp_ratio)+.000000001)
        else:
            fold_training.loc[data_idx, 'customer_virus_ratio_avg'] = 0
            fold_training.loc[data_idx, 'customer_virus_ratio_min'] = 0
            fold_training.loc[data_idx, 'customer_virus_ratio_max'] = 0
            fold_training.loc[data_idx, 'customer_virus_ratio_median'] = 0
            fold_training.loc[data_idx, 'customer_virus_ratio_std'] = 0
            fold_training.loc[data_idx, 'customer_virus_ratio_rstd'] = 0
            
        temp_ratio = []
        for product_id in unique_products:
            try:
                found_ratio = product_virus_ratio_df_group.get_group(product_id).iloc[0]
                if found_ratio['total_event_count'] >= 3:
                    temp_ratio.append(found_ratio['ratio'])
            except:
                temp_ratio.append(0)
                
        if len(temp_ratio) > 0:        
            fold_training.loc[data_idx, 'product_virus_ratio_avg'] = np.mean(temp_ratio)
            fold_training.loc[data_idx, 'product_virus_ratio_min'] = np.min(temp_ratio)
            fold_training.loc[data_idx, 'product_virus_ratio_max'] = np.max(temp_ratio)
            fold_training.loc[data_idx, 'product_virus_ratio_median'] = np.median(temp_ratio)
            fold_training.loc[data_idx, 'product_virus_ratio_std'] = np.std(temp_ratio)
            fold_training.loc[data_idx, 'product_virus_ratio_rstd'] = np.std(temp_ratio) / (np.mean(temp_ratio)+.000000001)
        else:
            fold_training.loc[data_idx, 'product_virus_ratio_avg'] = 0
            fold_training.loc[data_idx, 'product_virus_ratio_min'] = 0
            fold_training.loc[data_idx, 'product_virus_ratio_max'] = 0
            fold_training.loc[data_idx, 'product_virus_ratio_median'] = 0
            fold_training.loc[data_idx, 'product_virus_ratio_std'] = 0
            fold_training.loc[data_idx, 'product_virus_ratio_rstd'] = 0
        
        customer_has_static_df = customer_file_count_df.iloc[temp_count_index, :]
        fold_training.loc[data_idx, 'customer_has_static_ratio'] = customer_has_static_df.shape[0] / len(unique_customers)
        customer_has_static_b3_df = customer_has_static_df[customer_has_static_df['total_event_count']>=3]
        fold_training.loc[data_idx, 'customer_has_static_b3_ratio'] = customer_has_static_b3_df.shape[0] / len(unique_customers)

        fold_training.loc[data_idx, 'customer_has_static_count'] = customer_has_static_df.shape[0]
        fold_training.loc[data_idx, 'customer_has_static_b3_count'] = customer_has_static_b3_df.shape[0]

        if customer_has_static_df.shape[0] > 0:
            fold_training.loc[data_idx, 'customer_virus_ratio_wavg'] = np.average(customer_has_static_df['ratio'], weights=customer_has_static_df['total_event_count'])            
            fold_training.loc[data_idx, 'customer_virus_zero_count'] = np.sum(customer_has_static_df['virus_event_count']==0)
            fold_training.loc[data_idx, 'customer_virus_zero_ratio'] = fold_training.loc[data_idx, 'customer_virus_zero_count'] / fold_training.loc[data_idx, 'affect_user_count']
            fold_training.loc[data_idx, 'customer_virus_zero_ratio2'] = fold_training.loc[data_idx, 'customer_virus_zero_count'] / fold_training.loc[data_idx, 'customer_has_static_count']
        
#         print(fold_training.loc[data_idx])
#         break
    
    print(fold_training.head())
    training_five_folds.append(fold_training)
    

In [ ]:
training_df = pd.concat([training_five_folds[0], training_five_folds[1], training_five_folds[2], training_five_folds[3], training_five_folds[4]])

In [ ]:
test_df['affect_user_count'] = None
test_df['total_event_count'] = None
test_df['each_user_event_avg'] = None
test_df['each_user_event_std'] = None
test_df['each_user_event_rstd'] = None

test_df['first_time_occur'] = None
test_df['last_time_occur'] = None
test_df['time_duration'] = None

test_df['first_24h_count'] = None
test_df['first_24h_ratio'] = None
test_df['first_24h_affect_user_count'] = None
test_df['first_24h_affect_user_ratio'] = None
test_df['first_24h_each_user_event_avg'] = None

for i in range(0, 10):
    test_df['bin_'+str(i)+'_count'] = None
    test_df['bin_'+str(i)+'_ratio'] = None
    test_df['bin_'+str(i)+'_affect_user_count'] = None
    test_df['bin_'+str(i)+'_affect_user_ratio'] = None
    test_df['bin_'+str(i)+'_each_user_event_avg'] = None

test_df['customer_spread_time_mean'] = None
test_df['customer_spread_time_std'] = None
test_df['customer_spread_time_rstd'] = None

test_df['event_diff_time_mean'] = None
test_df['event_diff_time_median'] = None
test_df['event_diff_time_std'] = None
test_df['event_diff_time_rstd'] = None

test_df['customer_virus_ratio_avg'] = None
test_df['customer_virus_ratio_min'] = None
test_df['customer_virus_ratio_max'] = None
test_df['customer_virus_ratio_median'] = None
test_df['customer_virus_ratio_std'] = None
test_df['customer_virus_ratio_rstd'] = None  

test_df['product_virus_ratio_avg'] = None
test_df['product_virus_ratio_min'] = None
test_df['product_virus_ratio_max'] = None
test_df['product_virus_ratio_median'] = None
test_df['product_virus_ratio_std'] = None
test_df['product_virus_ratio_rstd'] = None  

test_df['customer_has_static_ratio'] = None
test_df['customer_has_static_b3_ratio'] = None
test_df['customer_has_static_count'] = None
test_df['customer_has_static_b3_count'] = None
test_df['customer_virus_ratio_wavg'] = None
test_df['customer_virus_zero_count'] = None  
test_df['customer_virus_zero_ratio'] = None  
test_df['customer_virus_zero_ratio2'] = None  

test_df['hour_ratio_mean'] = None
test_df['hour_ratio_median'] = None
test_df['hour_ratio_std'] = None
test_df['hour_ratio_rstd'] = None
test_df['hour_ratio_max'] = None
test_df['hour_ratio_min'] = None
test_df['hour_occupy_ratio'] = None
test_df['hour_mean'] = None

test_df['dayofweek_ratio_mean'] = None
test_df['dayofweek_ratio_median'] = None
test_df['dayofweek_ratio_std'] = None
test_df['dayofweek_ratio_rstd'] = None
test_df['dayofweek_ratio_max'] = None
test_df['dayofweek_ratio_min'] = None
test_df['dayofweek_occupy_ratio'] = None
test_df['dayofweek_mean'] = None

test_df['bin_ratio_mean'] = None
test_df['bin_ratio_median'] = None
test_df['bin_ratio_std'] = None
test_df['bin_ratio_rstd'] = None
test_df['bin_ratio_max'] = None
test_df['bin_ratio_min'] = None
test_df['bin_occupy_ratio'] = None

test_df['event_diff_time_mean2'] = None
test_df['event_diff_time_median2'] = None
test_df['event_diff_time_std2'] = None
test_df['event_diff_time_rstd2'] = None
test_df['event_diff_time_max2'] = None
test_df['event_diff_time_min2'] = None
test_df['event_diff_time_lessone_ratio2'] = None

test_df['all_event_diff_time_mean'] = None
test_df['all_event_diff_time_median'] = None
test_df['all_event_diff_time_std'] = None
test_df['all_event_diff_time_rstd'] = None
test_df['all_event_diff_time_max'] = None
test_df['all_event_diff_time_min'] = None
test_df['all_event_diff_time_lessone_ratio'] = None

for idx, data in tqdm(test_df.iterrows(), total=len(test_df)):
    temp_df = pd.read_csv('file_log/'+data['FileID']+'.csv', header=None, names=['FileID', 'CustomerID', 'QueryTS', 'ProductID'])
    total_event_count = len(temp_df)
    test_df.loc[idx, 'affect_user_count'] = len(temp_df['CustomerID'].unique())
    test_df.loc[idx, 'total_event_count'] = total_event_count
    test_df.loc[idx, 'each_user_event_avg'] = test_df.loc[idx, 'total_event_count'] / test_df.loc[idx, 'affect_user_count']
    test_df.loc[idx, 'each_user_event_std'] = temp_df.groupby('CustomerID').count()['QueryTS'].std()
    test_df.loc[idx, 'each_user_event_rstd'] = test_df.loc[idx, 'each_user_event_std'] / test_df.loc[idx, 'each_user_event_avg']
    
    time_start = temp_df['QueryTS'].min()
    time_end = temp_df['QueryTS'].max()
    time_range = time_end - time_start
    
    test_df.loc[idx, 'first_time_occur'] = time_start
    test_df.loc[idx, 'last_time_occur'] = time_end
    test_df.loc[idx, 'time_duration'] = time_range
    
    the_flag = ( (temp_df['QueryTS']>=time_start)&(temp_df['QueryTS']<=time_start+86400) )
    the_24h_count = np.sum(the_flag)
    test_df.loc[idx, 'first_24h_count'] = the_24h_count
    test_df.loc[idx, 'first_24h_ratio'] = the_24h_count / total_event_count
    test_df.loc[idx, 'first_24h_affect_user_count'] = len(temp_df[the_flag]['CustomerID'].unique())
    test_df.loc[idx, 'first_24h_affect_user_ratio'] = test_df.loc[idx, 'first_24h_affect_user_count'] / test_df.loc[idx, 'affect_user_count']
    test_df.loc[idx, 'first_24h_each_user_event_avg'] = the_24h_count / (test_df.loc[idx, 'first_24h_affect_user_count']+.000000001)
    
    time_step = time_range/10.0
    for i in range(0, 10):
        the_flag = ( (temp_df['QueryTS']>=time_start+time_step*i)&(temp_df['QueryTS']<time_start+time_step*(i+1)+0.5) )
        the_bin_count = np.sum(the_flag)
        test_df.loc[idx, 'bin_'+str(i)+'_count'] = the_bin_count
        test_df.loc[idx, 'bin_'+str(i)+'_ratio'] = the_bin_count / total_event_count
        test_df.loc[idx, 'bin_'+str(i)+'_affect_user_count'] = len(temp_df[the_flag]['CustomerID'].unique())
        test_df.loc[idx, 'bin_'+str(i)+'_affect_user_ratio'] = test_df.loc[idx, 'bin_'+str(i)+'_affect_user_count'] / test_df.loc[idx, 'affect_user_count']
        test_df.loc[idx, 'bin_'+str(i)+'_each_user_event_avg'] = the_bin_count / (test_df.loc[idx, 'bin_'+str(i)+'_affect_user_count']+.000000001)
    
    the_static_df = temp_df.groupby('CustomerID').min().sort_values(['QueryTS'], ascending=True)['QueryTS'].diff().fillna(0)
    test_df.loc[idx, 'customer_spread_time_mean'] = the_static_df.mean()
    test_df.loc[idx, 'customer_spread_time_std'] = the_static_df.std()
    test_df.loc[idx, 'customer_spread_time_rstd'] = test_df.loc[idx, 'customer_spread_time_std'] / (test_df.loc[idx, 'customer_spread_time_mean']+.000000001)
    
    the_static_df = temp_df.sort_values(['CustomerID', 'QueryTS'], ascending=True).groupby(['CustomerID'])['QueryTS'].diff().fillna(0)
    test_df.loc[idx, 'event_diff_time_mean'] = the_static_df.mean()
    test_df.loc[idx, 'event_diff_time_median'] = the_static_df.median()
    test_df.loc[idx, 'event_diff_time_std'] = the_static_df.std()
    test_df.loc[idx, 'event_diff_time_rstd'] = test_df.loc[idx, 'event_diff_time_std'] / (test_df.loc[idx, 'event_diff_time_mean']+.000000001)
    
    unique_customers = temp_df['CustomerID'].unique().tolist()
    temp_ratio = []
    temp_count_index = []
    for customer_id in unique_customers:
        try:
            found_ratio = customer_virus_ratio_df_group.get_group(customer_id).iloc[0]
            if found_ratio['total_event_count'] >= 3:
                temp_ratio.append(found_ratio['ratio'])
        except:
            temp_ratio.append(0)
            
        try:
            found_count_index = bisect.bisect(customer_file_count_array, customer_id)
            if customer_file_count_array[found_count_index-1] == customer_id:
                temp_count_index.append(found_count_index-1)
        except:
            pass
        
    if len(temp_ratio) > 0:        
        test_df.loc[idx, 'customer_virus_ratio_avg'] = np.mean(temp_ratio)
        test_df.loc[idx, 'customer_virus_ratio_min'] = np.min(temp_ratio)
        test_df.loc[idx, 'customer_virus_ratio_max'] = np.max(temp_ratio)
        test_df.loc[idx, 'customer_virus_ratio_median'] = np.median(temp_ratio)
        test_df.loc[idx, 'customer_virus_ratio_std'] = np.std(temp_ratio)
        test_df.loc[idx, 'customer_virus_ratio_rstd'] = np.std(temp_ratio) / (np.mean(temp_ratio)+.000000001)
    else:
        test_df.loc[idx, 'customer_virus_ratio_avg'] = 0
        test_df.loc[idx, 'customer_virus_ratio_min'] = 0
        test_df.loc[idx, 'customer_virus_ratio_max'] = 0
        test_df.loc[idx, 'customer_virus_ratio_median'] = 0
        test_df.loc[idx, 'customer_virus_ratio_std'] = 0
        test_df.loc[idx, 'customer_virus_ratio_rstd'] = 0
    
    unique_products = temp_df['ProductID'].unique().tolist()
    temp_ratio = []
    for product_id in unique_products:
        try:
            found_ratio = product_virus_ratio_df_group.get_group(product_id).iloc[0]
            if found_ratio['total_event_count'] >= 3:
                temp_ratio.append(found_ratio['ratio'])
        except:
            temp_ratio.append(0)

    if len(temp_ratio) > 0:
        test_df.loc[idx, 'product_virus_ratio_avg'] = np.mean(temp_ratio)
        test_df.loc[idx, 'product_virus_ratio_min'] = np.min(temp_ratio)
        test_df.loc[idx, 'product_virus_ratio_max'] = np.max(temp_ratio)
        test_df.loc[idx, 'product_virus_ratio_median'] = np.median(temp_ratio)
        test_df.loc[idx, 'product_virus_ratio_std'] = np.std(temp_ratio)
        test_df.loc[idx, 'product_virus_ratio_rstd'] = np.std(temp_ratio) / (np.mean(temp_ratio)+.000000001)
    else:
        test_df.loc[idx, 'product_virus_ratio_avg'] = 0
        test_df.loc[idx, 'product_virus_ratio_min'] = 0
        test_df.loc[idx, 'product_virus_ratio_max'] = 0
        test_df.loc[idx, 'product_virus_ratio_median'] = 0
        test_df.loc[idx, 'product_virus_ratio_std'] = 0
        test_df.loc[idx, 'product_virus_ratio_rstd'] = 0
        
    customer_has_static_df = customer_file_count_df.iloc[temp_count_index, :]
    test_df.loc[idx, 'customer_has_static_ratio'] = customer_has_static_df.shape[0] / len(unique_customers)
    customer_has_static_b3_df = customer_has_static_df[customer_has_static_df['total_event_count']>=3]
    test_df.loc[idx, 'customer_has_static_b3_ratio'] = customer_has_static_b3_df.shape[0] / len(unique_customers)

    test_df.loc[idx, 'customer_has_static_count'] = customer_has_static_df.shape[0]
    test_df.loc[idx, 'customer_has_static_b3_count'] = customer_has_static_b3_df.shape[0]

    if customer_has_static_df.shape[0] > 0:
        test_df.loc[idx, 'customer_virus_ratio_wavg'] = np.average(customer_has_static_df['ratio'], weights=customer_has_static_df['total_event_count'])            
        test_df.loc[idx, 'customer_virus_zero_count'] = np.sum(customer_has_static_df['virus_event_count']==0)
        test_df.loc[idx, 'customer_virus_zero_ratio'] = test_df.loc[idx, 'customer_virus_zero_count'] / test_df.loc[idx, 'affect_user_count']
        test_df.loc[idx, 'customer_virus_zero_ratio2'] = test_df.loc[idx, 'customer_virus_zero_count'] / test_df.loc[idx, 'customer_has_static_count']

    temp_df['datetime'] = pd.to_datetime(temp_df['QueryTS'], unit='s')
    temp_df['hour'] = temp_df['datetime'].dt.hour
    temp_df['dayofweek'] = temp_df['datetime'].dt.dayofweek
    
    hour_ratio_df = temp_df[['FileID', 'hour']].groupby('hour').count()/temp_df.shape[0]
    test_df.loc[idx, 'hour_ratio_mean'] = hour_ratio_df['FileID'].mean()
    test_df.loc[idx, 'hour_ratio_median'] = hour_ratio_df['FileID'].median()
    test_df.loc[idx, 'hour_ratio_std'] = hour_ratio_df['FileID'].std()
    test_df.loc[idx, 'hour_ratio_rstd'] = hour_ratio_df['FileID'].std() / (hour_ratio_df['FileID'].mean()+.000000001)
    test_df.loc[idx, 'hour_ratio_max'] = hour_ratio_df['FileID'].max()
    test_df.loc[idx, 'hour_ratio_min'] = hour_ratio_df['FileID'].min()
    test_df.loc[idx, 'hour_occupy_ratio'] = temp_df[['FileID', 'hour']].groupby('hour').count().shape[0]/24.0
    test_df.loc[idx, 'hour_mean'] = temp_df['hour'].mean()
    
    dayofweek_ratio_df = temp_df[['FileID', 'dayofweek']].groupby('dayofweek').count()/temp_df.shape[0]
    test_df.loc[idx, 'dayofweek_ratio_mean'] = dayofweek_ratio_df['FileID'].mean()
    test_df.loc[idx, 'dayofweek_ratio_median'] = dayofweek_ratio_df['FileID'].median()
    test_df.loc[idx, 'dayofweek_ratio_std'] = dayofweek_ratio_df['FileID'].std()
    test_df.loc[idx, 'dayofweek_ratio_rstd'] = dayofweek_ratio_df['FileID'].std() / (dayofweek_ratio_df['FileID'].mean()+.000000001)
    test_df.loc[idx, 'dayofweek_ratio_max'] = dayofweek_ratio_df['FileID'].max()
    test_df.loc[idx, 'dayofweek_ratio_min'] = dayofweek_ratio_df['FileID'].min()
    test_df.loc[idx, 'dayofweek_occupy_ratio'] = temp_df[['FileID', 'dayofweek']].groupby('dayofweek').count().shape[0]/7.0
    test_df.loc[idx, 'dayofweek_mean'] = temp_df['dayofweek'].mean()
    
    bin_ratio_array = test_df.loc[idx, ['bin_0_ratio', 'bin_1_ratio', 'bin_2_ratio', 'bin_3_ratio', 'bin_4_ratio', \
                            'bin_5_ratio', 'bin_6_ratio', 'bin_7_ratio', 'bin_8_ratio', 'bin_9_ratio']].values
    test_df.loc[idx, 'bin_ratio_mean'] = np.mean(bin_ratio_array)
    test_df.loc[idx, 'bin_ratio_median'] = np.median(bin_ratio_array)
    test_df.loc[idx, 'bin_ratio_std'] = np.std(bin_ratio_array)
    test_df.loc[idx, 'bin_ratio_rstd'] = np.std(bin_ratio_array) / (np.mean(bin_ratio_array)+.000000001)
    test_df.loc[idx, 'bin_ratio_max'] = np.max(bin_ratio_array)
    test_df.loc[idx, 'bin_ratio_min'] = np.min(bin_ratio_array)
    test_df.loc[idx, 'bin_occupy_ratio'] = np.sum(bin_ratio_array>0)/10.0
    
    the_static_df = temp_df.sort_values(['CustomerID', 'QueryTS'], ascending=True).groupby(['CustomerID'])['QueryTS'].diff().dropna()
    if the_static_df.shape[0] > 0:
        test_df.loc[idx, 'event_diff_time_mean2'] = the_static_df.mean()
        test_df.loc[idx, 'event_diff_time_median2'] = the_static_df.median()
        test_df.loc[idx, 'event_diff_time_std2'] = the_static_df.std()
        test_df.loc[idx, 'event_diff_time_rstd2'] = test_df.loc[idx, 'event_diff_time_std2'] / (test_df.loc[idx, 'event_diff_time_mean2']+.000000001)
        test_df.loc[idx, 'event_diff_time_max2'] = the_static_df.max()
        test_df.loc[idx, 'event_diff_time_min2'] = the_static_df.min()
        test_df.loc[idx, 'event_diff_time_lessone_ratio2'] = np.sum(the_static_df<=1) / len(the_static_df)
    
    the_static_df = temp_df.sort_values(['QueryTS'], ascending=True)['QueryTS'].diff().dropna()
    if the_static_df.shape[0] > 0:
        test_df.loc[idx, 'all_event_diff_time_mean'] = the_static_df.mean()
        test_df.loc[idx, 'all_event_diff_time_median'] = the_static_df.median()
        test_df.loc[idx, 'all_event_diff_time_std'] = the_static_df.std()
        test_df.loc[idx, 'all_event_diff_time_rstd'] = test_df.loc[idx, 'all_event_diff_time_std'] / (test_df.loc[idx, 'all_event_diff_time_mean']+.000000001)
        test_df.loc[idx, 'all_event_diff_time_max'] = the_static_df.max()
        test_df.loc[idx, 'all_event_diff_time_min'] = the_static_df.min()
        test_df.loc[idx, 'all_event_diff_time_lessone_ratio'] = np.sum(the_static_df<=1) / len(the_static_df)
        
#     print(test_df.loc[idx])
# #     print(temp_df)
# #     print(len(temp_df['CustomerID'].unique()))
# #     print(data)
#     if idx==2:
#         break

## 訓練模型

In [ ]:
layer1_df = pd.DataFrame()

In [ ]:
useful_columns = ['bin_0_affect_user_ratio', 'first_24h_each_user_event_avg', 'bin_ratio_max', \
                  'customer_has_static_b3_count', 'hour_occupy_ratio', 'bin_9_affect_user_ratio', \
                  'dayofweek_ratio_std', 'event_diff_time_rstd', 'each_user_event_std', \
                  'event_diff_time_lessone_ratio2', 'bin_7_affect_user_ratio', 'first_24h_affect_user_ratio', \
                  'all_event_diff_time_mean', 'product_virus_ratio_max', 'customer_virus_zero_count', \
                  'customer_spread_time_mean', 'customer_virus_ratio_rstd', 'first_24h_affect_user_count', \
                  'hour_ratio_max', 'hour_ratio_std', 'customer_has_static_b3_ratio', 'first_24h_ratio', \
                  'customer_has_static_ratio', 'customer_has_static_count', 'event_diff_time_median', \
                  'hour_ratio_mean', 'first_24h_count', 'customer_virus_ratio_median', 'event_diff_time_median2', \
                  'all_event_diff_time_median', 'customer_virus_ratio_std', 'customer_virus_ratio_avg', \
                  'customer_virus_ratio_max', 'each_user_event_avg', 'customer_virus_zero_ratio', \
                  'customer_virus_zero_ratio2', 'customer_virus_ratio_wavg']

In [ ]:
import lightgbm as lgb

In [ ]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'l2', 'auc'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.6,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0,
    'lambda_l2': 5.0,
    'min_gain_to_split': 0,
    'min_data_in_leaf': 8,
    'max_depth': 15
}

lgb_train = lgb.Dataset(training_df[useful_columns].values, training_df['label'].values)
lgb_eval = lgb.Dataset(training_df[useful_columns].values, training_df['label'].values, reference=lgb_train)
gbm = lgb.train(params,
            lgb_train,
            num_boost_round=105,
            valid_sets=lgb_eval)

fpr, tpr, thresholds = roc_curve(training_df['label'], gbm.predict(training_df[useful_columns].values, num_iteration=gbm.best_iteration))
print(auc(fpr, tpr))

In [ ]:
layer1_df['lgbm'] = gbm.predict(training_df[useful_columns].values, num_iteration=gbm.best_iteration)

In [ ]:
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn import preprocessing

In [ ]:
scaler = preprocessing.StandardScaler().fit(training_df[useful_columns])

In [ ]:
predict_results = []
clssfis = []
for idx, fold in enumerate(five_folds):
    merge_folds = fold[0]
    fold_training = training_df[training_df['fold'].isin(merge_folds)]

    clssfi = MLPClassifier(random_state=0, hidden_layer_sizes=(10, 3, 3), alpha=0.2)
    clssfi.fit(scaler.transform(fold_training[useful_columns]), fold_training['label'])
    fpr, tpr, thresholds = roc_curve(fold_training['label'], clssfi.predict_proba(scaler.transform(fold_training[useful_columns]))[:, 1])
    print(auc(fpr, tpr))
    
    clssfis.append(clssfi)
    predict_results.append(clssfi.predict_proba(scaler.transform(training_df[useful_columns]))[:, 1])

In [ ]:
fpr, tpr, thresholds = roc_curve(training_df['label'], np.mean(predict_results, axis=0))
print(auc(fpr, tpr))
layer1_df['nn'] = np.mean(predict_results, axis=0)

In [ ]:
fpr, tpr, thresholds = roc_curve(training_df['label'], layer1_df['nn'])
print(auc(fpr, tpr))
fpr, tpr, thresholds = roc_curve(training_df['label'], layer1_df['lgbm'])
print(auc(fpr, tpr))
fpr, tpr, thresholds = roc_curve(training_df['label'], layer1_df['nn']*0.4+layer1_df['lgbm']*0.6)
print(auc(fpr, tpr))

In [ ]:
layer1_df = pd.DataFrame()

In [ ]:
layer1_df['lgbm'] = gbm.predict(test_df[useful_columns].values, num_iteration=gbm.best_iteration)

In [ ]:
predict_results = []
for clssfi in clssfis:
    predict_results.append(clssfi.predict_proba(scaler.transform(test_df[useful_columns]))[:, 1])

In [ ]:
layer1_df['nn'] = np.mean(predict_results, axis=0)

In [ ]:
result_df = test_df.copy()
result_df['ans'] = layer1_df['nn']*0.4+layer1_df['lgbm']*0.6
result_df[['FileID', 'ans']].to_csv('df_log/228/result_0323.csv', index=False, header=False)